TO DO:
- Add option to force players into the team, i.e., optimize given certain player are already locked in

In [1]:
# previous gameweek
gameweek=37

In [2]:
import pandas as pd
import numpy as np
import json
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [3]:
# fetch player_dict 
with open('../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)

# Solver definition

In [4]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES

    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    captaincy_variables = [LpVariable(name=f'cap_{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'price'] for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'position']=='GK' else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'position']=='DEF' else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'position']=='MID' else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'position']=='FWD' else 0.0 for ix in df.index]
    teams = []
    for team in df['team'].unique():
        teams.append( [1.0 if df.loc[ix,'team']==team else 0.0 for ix in df.index] )

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # select 1 captain
    model += lpSum(captaincy_variables) == 1
    # captain must be one of the 11 players in the team
    for i in range(0,len(variables)):
        model += captaincy_variables[i] <= variables[i]
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    # max 3 players from any given team
    for team in teams:
        model += np.dot(team,variables) <= 3

    # OBJECTIVE
    # the second part doubles the captain's points
    model += np.dot(expected_points,variables) + np.dot(expected_points,captaincy_variables)

    # SOLVE OPTIMIZATION

    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [str(var) for var in model.variables() if var.value()==1]
    captain = [player for player in players if 'cap' in player]
    captain = int(captain[0].replace('cap_', ''))
    players = [int(player) for player in players if 'cap' not in player]
    dream_team = df.loc[players]
    cost = (dream_team['price']).sum()
    exp_points = dream_team[expected_column].sum() + df.loc[captain, expected_column]
    print(f'Cost: {cost}')
    print(f'Expected points per week: {exp_points}')
    
    display(dream_team[['position','name', 'team', 'price',expected_column]].sort_values('position'))
    
    return players

# Data

In [5]:
#projections = pd.read_csv(f'../point_projections/xP_projections_week{gameweek}.csv', index_col=0)
projections = pd.read_csv(f'../point_projections/collective_xP_projections_week{gameweek}.csv', index_col=0)
projections = projections.rename(columns={'xP':'projected_points'})
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std
0,Granit Xhaka,Arsenal,Wolves,1,38,3.510894,0.231156
1,Mohamed Elneny,Arsenal,Wolves,1,38,2.651292,0.162667
2,Rob Holding,Arsenal,Wolves,1,38,3.057946,0.103189
3,Thomas Partey,Arsenal,Wolves,1,38,2.775875,0.125040
4,Martin degaard,Arsenal,Wolves,1,38,5.386364,0.432688


In [6]:
# decay for future projected points to model uncertainty increasing over time
decay_coeff = 0.875
projections['projected_points_with_decay'] = ( projections['projected_points']*decay_coeff
                                              **(projections['gameweek']-(gameweek+1)) )
projections.head(5)

,name,team,opp_team,was_home,gameweek,projected_points,std,projected_points_with_decay
0,Granit Xhaka,Arsenal,Wolves,1,38,3.510894,0.231156,3.510894
1,Mohamed Elneny,Arsenal,Wolves,1,38,2.651292,0.162667,2.651292
2,Rob Holding,Arsenal,Wolves,1,38,3.057946,0.103189,3.057946
3,Thomas Partey,Arsenal,Wolves,1,38,2.775875,0.125040,2.775875
4,Martin degaard,Arsenal,Wolves,1,38,5.386364,0.432688,5.386364


In [7]:
fpl_df = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv', index_col=0)
fpl_df['name'] = fpl_df['name'].map(player_name_dict).fillna(fpl_df['name'])
fpl_df.head(5)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,100.0,100.0,84450,0,0,-2,2,1,3,5.3,...,2022-23,Granit Xhaka,MID,37,4.8,309020.4,2730,Nottingham Forest,2023-05-20T16:30:00Z,0.0
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,...,2022-23,Mohamed Elneny,MID,37,4.1,68671.2,304,Nottingham Forest,2023-05-20T16:30:00Z,0.0
2,NaN,NaN,156074,0,0,-3,3,0,2,0.7,...,2022-23,Rob Holding,DEF,37,4.2,22890.4,-112,Nottingham Forest,2023-05-20T16:30:00Z,0.0
3,100.0,100.0,167199,0,0,-3,3,0,3,1.5,...,2022-23,Thomas Partey,MID,37,4.7,45780.8,114,Nottingham Forest,2023-05-20T16:30:00Z,0.0
4,100.0,100.0,184029,1,-1,4,-4,5,3,7.7,...,2022-23,Martin degaard,MID,37,6.9,3685354.4,108072,Nottingham Forest,2023-05-20T16:30:00Z,0.0


In [8]:
# choose time horizon for optimization
#horizon = 10

# create dataframe to be given as input for optimization
#df = projections[projections['gameweek']<=gameweek+horizon].groupby(by=['name']).mean()\
#                                                        [['projected_points','projected_points_with_decay', 'std']] 

# OPTIMIZE FOR GIVEN GAMEWEEKS
my_gameweeks = [38]
df = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).sum()\
                                                        [['projected_points','projected_points_with_decay']]
df['std'] = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).mean()\
                                                        [['std']]
df['number_of_games'] = (projections[projections['gameweek'].isin(my_gameweeks)]
                                     .groupby('name').count()['std'])

#df = df.to_frame()
df = df.reset_index()
#df = df.rename(columns={'projected_points':'projected_points_per_game'})
#df['team'] = [fpl_df.loc[fpl_df['name']==name, 'team'].values[0] for name in df['name']]
df['price'] = [fpl_df.loc[fpl_df['name']==name, 'value'].values[0] for name in df['name']]
df['position'] = [fpl_df.loc[fpl_df['name']==name, 'element_type'].values[0] for name in df['name']]
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
df['position'] = df['position'].map(position_dict)
df['position'] = pd.Categorical(df['position'], ['GK','DEF','MID','FWD'])
df['team'] = [projections.loc[projections['name']==name, 'team'].head(1).values[0] for name in df['name']]
df['minutes'] = [fpl_df.loc[fpl_df['name']==name, 'minutes'].values[0] for name in df['name']]
# re-order columns
df = df[['name', 'team', 'position', 'price', 'minutes', 'number_of_games', 'projected_points', 'projected_points_with_decay', 'std']]
df.head(5)

/var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ipykernel_27320/4132594464.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).sum()\
/var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/ipykernel_27320/4132594464.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df['std'] = projections[projections['gameweek'].isin(my_gameweeks)].groupby(by=['name']).mean()\


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std
0,Aaron Cresswell,West Ham,DEF,4.7,2165,1,2.888555,2.888555,0.112888
1,Aaron Hickey,Brentford,DEF,4.9,1849,1,2.030285,2.030285,0.112578
2,Aaron Ramsdale,Arsenal,GK,4.8,3330,1,4.012417,4.012417,0.091788
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,1432,1,3.663708,3.663708,0.148226
4,Abdoulaye Doucoure,Everton,MID,5.3,1437,1,2.911545,2.911545,0.166233


In [9]:
# remove players who do not have enough minutes played
cutoff = 450
df = df[df['minutes']>=cutoff]

# edit some player prices according to my selling price
df.loc[df['name']=='Erling Haaland', 'price'] = 11.8
df.loc[df['name']=='Martinelli', 'price'] = 6.3

# Optimization

In [10]:
team_value = 102
money_in_the_bank = 0.2
allowed_bench_value = 17.5
max_price = team_value + money_in_the_bank - allowed_bench_value

## Without decay

In [ ]:
players = optimize(df, max_price, 'projected_points_per_game')

In [ ]:
ban = []
# ban some cheap keepers in good teams who don't actually play
cheap_keepers = ['Scott Carson', 'Zack Steffen', 'Jason Steele', 'Caoimhin Kelleher', 'Wayne Hennessey', 
                 'Karl Darlow', 'Darren Randolph']
cheap_keepers_ix = df[df['name'].isin(cheap_keepers)].index
ban = ban + list(cheap_keepers_ix)
# ban given players
ban_field = ['Marcos Alonso', 'Ben Chilwell', 'Riyad Mahrez', 'Darwin Nunez', 'Aymeric Laporte', 'Emil Krafth', 
             'Sergio Gomez', 'Nathaniel Phillips']
ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [ ]:
players = optimize(my_df, max_price, 'projected_points_per_game')

## With decay

In [ ]:
#players = optimize(df, max_price, 'projected_points_with_decay')

In [11]:
ban = []

# ban given players
ban_field = ['Aymeric Laporte', 'Riyad Mahrez', 'Solly March']
#ban_field = []

ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [12]:
players = optimize(my_df, max_price, 'projected_points_with_decay')

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9c9669f482e34606b977b6de6db9e8cc-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/9c9669f482e34606b977b6de6db9e8cc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 411 COLUMNS
At line 5495 RHS
At line 5902 BOUNDS
At line 6657 ENDATA
Problem MODEL has 406 rows, 754 columns and 2821 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 66.0227 - 0.01 seconds
Cgl0004I processed model has 404 rows, 754 columns (754 integer (754 of which binary)) and 2639 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.571429
Cbc0038I Solution found of -65.9779
Cbc0038I Before mini branch and bound, 752 integers at bound fixed and 0 continuous
Cbc0038I F

,position,name,team,price,projected_points_with_decay
490,GK,Sam Johnstone,Crystal Palace,4.4,3.778704
29,DEF,Andrew Robertson,Liverpool,6.8,4.670727
309,DEF,Kieran Trippier,Newcastle Utd,6.0,4.459035
539,DEF,Trent Alexander-Arnold,Liverpool,7.8,4.840486
369,MID,Martin degaard,Arsenal,6.9,5.386364
38,MID,Antony,Manchester Utd,7.3,5.452375
395,MID,Mohamed Salah,Liverpool,13.1,7.898432
68,MID,Bruno Fernandes,Manchester Utd,9.5,6.055821
71,MID,Bukayo Saka,Arsenal,8.0,5.688718
14,FWD,Aleksandar Mitrovic,Fulham,6.6,4.584111


# Sensitivity analysis

Here we add small random Gaussian noise to each players' projected points and run the optimization with these new estimates. We repeat the process n times and keep a list of how many times each player appeared in the optimal team.

In [13]:
ban = []

# ban given players
ban_field = ['Aymeric Laporte', 'Riyad Mahrez', 'Solly March', 'Cristiano Ronaldo']
#ban_field = []

ban_field_ix = df[df['name'].isin(ban_field)].index
ban = ban + list(ban_field_ix)
my_df = df.drop(ban).copy()

In [14]:
metric = 'projected_points_with_decay'
# set the standard deviation of the Gaussian noise to be a fraction of each players' projected points
#sigma_factor = 0.1
simulation_rounds = 100
results = []
for i in range(0,simulation_rounds):
    df_with_noise = my_df.copy()
    #df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise[metric]*sigma_factor)
    df_with_noise[metric] = df_with_noise[metric] + np.random.normal(0, df_with_noise['std'])*df_with_noise['number_of_games']
    players = optimize(df_with_noise, max_price, metric)
    results.append(players)

Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/18a30cf51c684c6a91f42097c97c9808-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/18a30cf51c684c6a91f42097c97c9808-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 410 COLUMNS
At line 5480 RHS
At line 5886 BOUNDS
At line 6639 ENDATA
Problem MODEL has 405 rows, 752 columns and 2813 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 71.7004 - 0.01 seconds
Cgl0004I processed model has 403 rows, 752 columns (752 integer (752 of which binary)) and 2632 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.3
Cbc0038I Pass   1: suminf.    0.15000 (2) obj. -71.5653 iterations 9
Cbc0038I Solution found of -68.2703
Cbc0038I Before mini branch an

,position,name,team,price,projected_points_with_decay
490,GK,Sam Johnstone,Crystal Palace,4.4,3.955319
29,DEF,Andrew Robertson,Liverpool,6.8,4.558657
309,DEF,Kieran Trippier,Newcastle Utd,6.0,4.568574
539,DEF,Trent Alexander-Arnold,Liverpool,7.8,4.666638
369,MID,Martin degaard,Arsenal,6.9,5.943959
38,MID,Antony,Manchester Utd,7.3,5.526900
395,MID,Mohamed Salah,Liverpool,13.1,8.951060
68,MID,Bruno Fernandes,Manchester Utd,9.5,8.117889
71,MID,Bukayo Saka,Arsenal,8.0,6.531428
14,FWD,Aleksandar Mitrovic,Fulham,6.6,4.733724


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/10a8a70089f54a17afba97472760f6fe-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/10a8a70089f54a17afba97472760f6fe-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 410 COLUMNS
At line 5480 RHS
At line 5886 BOUNDS
At line 6639 ENDATA
Problem MODEL has 405 rows, 752 columns and 2813 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 69.5815 - 0.00 seconds
Cgl0004I processed model has 403 rows, 752 columns (752 integer (752 of which binary)) and 2632 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.125
Cbc0038I Pass   1: suminf.    0.06452 (2) obj. -69.4678 iterations 12
Cbc0038I Solution found of -63.654
Cbc0038I Before mini branch 

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.5,4.272141
29,DEF,Andrew Robertson,Liverpool,6.8,4.566062
309,DEF,Kieran Trippier,Newcastle Utd,6.0,4.518289
559,DEF,William Saliba,Arsenal,4.9,4.341507
109,MID,Dango Ouattara,Bournemouth,5.0,4.797907
369,MID,Martin degaard,Arsenal,6.9,5.431424
38,MID,Antony,Manchester Utd,7.3,5.171582
395,MID,Mohamed Salah,Liverpool,13.1,8.163455
68,MID,Bruno Fernandes,Manchester Utd,9.5,8.579011
180,FWD,Gabriel Jesus,Arsenal,8.1,5.222612


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/b7130b9cf92542e8afb644ec08ed0a7f-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/b7130b9cf92542e8afb644ec08ed0a7f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 410 COLUMNS
At line 5480 RHS
At line 5886 BOUNDS
At line 6639 ENDATA
Problem MODEL has 405 rows, 752 columns and 2813 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 67.853 - 0.00 seconds
Cgl0004I processed model has 403 rows, 752 columns (752 integer (752 of which binary)) and 2632 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.8
Cbc0038I Solution found of -67.64
Cbc0038I Before mini branch and bound, 750 integers at bound fixed and 0 continuous
Cbc0038I Full prob

...

,position,name,team,price,projected_points_with_decay
490,GK,Sam Johnstone,Crystal Palace,4.4,4.016900
29,DEF,Andrew Robertson,Liverpool,6.8,4.568717
309,DEF,Kieran Trippier,Newcastle Utd,6.0,4.821454
539,DEF,Trent Alexander-Arnold,Liverpool,7.8,4.757794
369,MID,Martin degaard,Arsenal,6.9,5.520298
38,MID,Antony,Manchester Utd,7.3,5.259327
395,MID,Mohamed Salah,Liverpool,13.1,8.194532
68,MID,Bruno Fernandes,Manchester Utd,9.5,5.659652
71,MID,Bukayo Saka,Arsenal,8.0,5.645039
14,FWD,Aleksandar Mitrovic,Fulham,6.6,4.469194


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/03a499298b80416eba84c2d0d405ee92-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/03a499298b80416eba84c2d0d405ee92-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 410 COLUMNS
At line 5480 RHS
At line 5886 BOUNDS
At line 6639 ENDATA
Problem MODEL has 405 rows, 752 columns and 2813 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 66.2795 - 0.00 seconds
Cgl0004I processed model has 403 rows, 752 columns (752 integer (752 of which binary)) and 2632 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.16667
Cbc0038I Solution found of -66.1991
Cbc0038I Before mini branch and bound, 746 integers at bound fixed and 0 continuous
Cbc0038I Fu

,position,name,team,price,projected_points_with_decay
547,GK,Vicente Guaita,Crystal Palace,4.4,3.885135
29,DEF,Andrew Robertson,Liverpool,6.8,4.517016
309,DEF,Kieran Trippier,Newcastle Utd,6.0,4.190483
345,DEF,Luke Shaw,Manchester Utd,5.3,4.220540
369,MID,Martin degaard,Arsenal,6.9,5.667947
38,MID,Antony,Manchester Utd,7.3,5.438624
395,MID,Mohamed Salah,Liverpool,13.1,7.327738
68,MID,Bruno Fernandes,Manchester Utd,9.5,7.667240
71,MID,Bukayo Saka,Arsenal,8.0,5.812366
139,FWD,Diogo Jota,Liverpool,8.6,4.695425


Welcome to the CBC MILP Solver 
Version: 2.9.0 
Build Date: Feb 12 2015 

command line - /Users/jaakkotoivonen/opt/anaconda3/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/a1fb6cca23b947d4bc70c943a9ca4d4e-pulp.mps max branch printingOptions all solution /var/folders/ml/jr8p__z97xx1jjb90rwg7hfm0000gn/T/a1fb6cca23b947d4bc70c943a9ca4d4e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 410 COLUMNS
At line 5480 RHS
At line 5886 BOUNDS
At line 6639 ENDATA
Problem MODEL has 405 rows, 752 columns and 2813 elements
Coin0008I MODEL read with 0 errors
Continuous objective value is 66.8228 - 0.00 seconds
Cgl0004I processed model has 403 rows, 752 columns (752 integer (752 of which binary)) and 2632 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.142857
Cbc0038I Pass   1: suminf.    0.02632 (2) obj. -66.7979 iterations 11
Cbc0038I Solution found of -63.5634
Cbc0038I Before mini bra

,position,name,team,price,projected_points_with_decay
21,GK,Alisson,Liverpool,5.5,4.305276
29,DEF,Andrew Robertson,Liverpool,6.8,4.751466
54,DEF,Ben White,Arsenal,5.0,4.187398
548,DEF,Victor Lindelof,Manchester Utd,4.1,3.835715
234,MID,James Maddison,Leicester City,7.8,5.342725
369,MID,Martin degaard,Arsenal,6.9,5.967071
395,MID,Mohamed Salah,Liverpool,13.1,8.528174
68,MID,Bruno Fernandes,Manchester Utd,9.5,5.403134
71,MID,Bukayo Saka,Arsenal,8.0,5.695204
14,FWD,Aleksandar Mitrovic,Fulham,6.6,4.814975


In [15]:
my_df['dream_team_appearances'] = [sum(team.count(player_id) for team in results) for player_id in my_df.index]
my_df.head(5)

,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
0,Aaron Cresswell,West Ham,DEF,4.7,2165,1,2.888555,2.888555,0.112888,0
1,Aaron Hickey,Brentford,DEF,4.9,1849,1,2.030285,2.030285,0.112578,0
2,Aaron Ramsdale,Arsenal,GK,4.8,3330,1,4.012417,4.012417,0.091788,15
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,1432,1,3.663708,3.663708,0.148226,6
4,Abdoulaye Doucoure,Everton,MID,5.3,1437,1,2.911545,2.911545,0.166233,0


In [16]:
print('Most appearances, all players:')
display(my_df.sort_values(by='dream_team_appearances', ascending=False).head(20))

Most appearances, all players:


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
395,Mohamed Salah,Liverpool,MID,13.1,3200,1,7.898432,7.898432,0.705574,99
38,Antony,Manchester Utd,MID,7.3,1803,1,5.452375,5.452375,0.389322,84
29,Andrew Robertson,Liverpool,DEF,6.8,2579,1,4.670727,4.670727,0.203420,79
180,Gabriel Jesus,Arsenal,FWD,8.1,1974,1,5.264741,5.264741,0.291895,76
71,Bukayo Saka,Arsenal,MID,8.0,3124,1,5.688718,5.688718,0.513934,76
68,Bruno Fernandes,Manchester Utd,MID,9.5,3234,1,6.055821,6.055821,1.538836,70
309,Kieran Trippier,Newcastle Utd,DEF,6.0,3252,1,4.459035,4.459035,0.400364,68
369,Martin degaard,Arsenal,MID,6.9,3058,1,5.386364,5.386364,0.432688,65
539,Trent Alexander-Arnold,Liverpool,DEF,7.8,2832,1,4.840486,4.840486,0.393989,61
14,Aleksandar Mitrovic,Fulham,FWD,6.6,1931,1,4.584111,4.584111,0.486573,50


In [17]:
print('GOALKEEPERS')
display(my_df[my_df['position']=='GK'].sort_values(by='dream_team_appearances', ascending=False).head(10))

GOALKEEPERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
21,Alisson,Liverpool,GK,5.5,3330,1,4.138495,4.138495,0.114769,25
124,David de Gea,Manchester Utd,GK,5.1,3330,1,3.935298,3.935298,0.180511,23
547,Vicente Guaita,Crystal Palace,GK,4.4,2430,1,3.759815,3.759815,0.160890,17
490,Sam Johnstone,Crystal Palace,GK,4.4,720,1,3.778704,3.778704,0.149593,17
2,Aaron Ramsdale,Arsenal,GK,4.8,3330,1,4.012417,4.012417,0.091788,15
248,Jason Steele,Brighton,GK,4.0,1260,1,3.046878,3.046878,0.272341,2
126,Dean Henderson,Nottingham Forest,GK,4.6,1620,1,3.388683,3.388683,0.165099,1
281,Jordan Pickford,Everton,GK,4.4,3240,1,3.449891,3.449891,0.127650,0
57,Bernd Leno,Fulham,GK,4.6,3150,1,2.779520,2.779520,0.197966,0
467,Robert Sanchez,Brighton,GK,4.4,2070,1,3.050539,3.050539,0.160756,0


In [18]:
print('DEFENDERS')
display(my_df[my_df['position']=='DEF'].sort_values(by='dream_team_appearances', ascending=False).head(20))

DEFENDERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
29,Andrew Robertson,Liverpool,DEF,6.8,2579,1,4.670727,4.670727,0.203420,79
309,Kieran Trippier,Newcastle Utd,DEF,6.0,3252,1,4.459035,4.459035,0.400364,68
539,Trent Alexander-Arnold,Liverpool,DEF,7.8,2832,1,4.840486,4.840486,0.393989,61
559,William Saliba,Arsenal,DEF,4.9,2415,1,4.215875,4.215875,0.123561,29
345,Luke Shaw,Manchester Utd,DEF,5.3,2552,1,3.887587,3.887587,0.184808,13
548,Victor Lindelof,Manchester Utd,DEF,4.1,1269,1,3.649562,3.649562,0.224987,12
549,Virgil van Dijk,Liverpool,DEF,6.6,2835,1,4.249728,4.249728,0.098831,11
179,Gabriel Dos Santos,Arsenal,DEF,5.2,3319,1,4.117754,4.117754,0.088494,8
138,Diogo Dalot,Manchester Utd,DEF,4.8,2062,1,3.849779,3.849779,0.173470,8
3,Aaron Wan-Bissaka,Manchester Utd,DEF,4.3,1432,1,3.663708,3.663708,0.148226,6


In [19]:
print('MIDFIELDERS')
display(my_df[my_df['position']=='MID'].sort_values(by='dream_team_appearances', ascending=False).head(20))

MIDFIELDERS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
395,Mohamed Salah,Liverpool,MID,13.1,3200,1,7.898432,7.898432,0.705574,99
38,Antony,Manchester Utd,MID,7.3,1803,1,5.452375,5.452375,0.389322,84
71,Bukayo Saka,Arsenal,MID,8.0,3124,1,5.688718,5.688718,0.513934,76
68,Bruno Fernandes,Manchester Utd,MID,9.5,3234,1,6.055821,6.055821,1.538836,70
369,Martin degaard,Arsenal,MID,6.9,3058,1,5.386364,5.386364,0.432688,65
234,James Maddison,Leicester City,MID,7.8,2389,1,5.162118,5.162118,0.566106,39
370,Martinelli,Arsenal,MID,6.3,2789,1,4.936330,4.936330,0.317336,22
363,Marcus Rashford,Manchester Utd,MID,7.2,2804,1,4.840024,4.840024,0.233832,18
134,Dennis Praet,Leicester City,MID,4.8,635,1,2.847232,2.847232,1.041398,8
557,Wilfried Zaha,Crystal Palace,MID,7.2,2289,1,4.736941,4.736941,0.291581,6


In [20]:
print('FORWARDS')
display(my_df[my_df['position']=='FWD'].sort_values(by='dream_team_appearances', ascending=False).head(20))

FORWARDS


,name,team,position,price,minutes,number_of_games,projected_points,projected_points_with_decay,std,dream_team_appearances
180,Gabriel Jesus,Arsenal,FWD,8.1,1974,1,5.264741,5.264741,0.291895,76
14,Aleksandar Mitrovic,Fulham,FWD,6.6,1931,1,4.584111,4.584111,0.486573,50
192,Harry Kane,Tottenham,FWD,11.4,3316,1,5.590032,5.590032,0.853748,48
19,Alexander Isak,Newcastle Utd,FWD,7.0,1433,1,4.202777,4.202777,0.282168,6
429,Ollie Watkins,Aston Villa,FWD,7.3,3039,1,4.134427,4.134427,0.330952,4
167,Erling Haaland,Manchester City,FWD,11.8,2767,1,4.463566,4.463566,0.724612,4
139,Diogo Jota,Liverpool,FWD,8.6,1035,1,4.388530,4.388530,0.252983,3
74,Callum Wilson,Newcastle Utd,FWD,7.3,1849,1,3.910159,3.910159,0.325730,2
35,Anthony Martial,Manchester Utd,FWD,6.3,947,1,3.957990,3.957990,0.326767,1
288,Julian Alvarez,Manchester City,FWD,6.1,1358,1,2.691094,2.691094,0.195213,0
